In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# Business Formation Statistics  
Census Bureau Product, find the page to download annual county-level statistics here: https://www.census.gov/econ/bfs/data/county.html  
Data Dictionary here: https://www.census.gov/econ/bfs/pdf/bfs_county_data_dictionary.pdf  
When downloaded, open the Excel workbook and delete the top two rows (the title and the notes) so that we are left with the row of column headers at the top. Copy-paste the content into an unformatted csv, and save in Data Downloads as "BusinessFormationStatistics_CountyAnnual_2005to2022.csv"

In [6]:
data = pd.read_csv('../../Data Downloads/BusinessFormationStatistics_CountyAnnual_2005to2022.csv', dtype = 'str')

In [7]:
data.head()

,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,BA2010,BA2011,BA2012,BA2013,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021,BA2022
0,AL,Autauga County,01001,01,001,335,348,348,367,288,304,278,242,282,290,291,294,313,384,394,559,769,697
1,AL,Baldwin County,01003,01,003,1986,2101,1975,1721,1505,1548,1553,1594,1532,1575,1735,1858,2114,2104,2317,2697,3392,3676
2,AL,Barbour County,01005,01,005,138,179,156,139,133,107,101,120,121,120,129,122,114,113,135,230,339,253
3,AL,Bibb County,01007,01,007,118,145,117,98,98,106,72,95,83,101,78,86,79,105,86,165,179,154
4,AL,Blount County,01009,01,009,315,323,297,306,247,247,263,228,225,228,218,254,281,299,312,344,464,414


In [11]:
rowsdict = {
    'BA2005': '2005', 
    'BA2006': '2006', 
    'BA2007': '2007', 
    'BA2008': '2008', 
    'BA2009': '2009', 
    'BA2010': '2010', 
    'BA2011': '2011', 
    'BA2012': '2012', 
    'BA2013': '2013', 
    'BA2014': '2014', 
    'BA2015': '2015', 
    'BA2016': '2016', 
    'BA2017': '2017', 
    'BA2018': '2018', 
    'BA2019': '2019', 
    'BA2020': '2020', 
    'BA2021': '2021',
    'BA2022': '2022',
    'fipscty': 'CountyFIPS', 
    'fipstate': 'StateFIPS', 
}
data = data.rename(columns = rowsdict)

In [13]:
data['GEO_ID'] = '0500000US' + data['StateFIPS'] + data['CountyFIPS']

In [14]:
data['County Option Two'] = data['County'] + ', ' + data['State']

In [15]:
data.head()

,State,County,County Code,StateFIPS,CountyFIPS,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,GEO_ID,County Option Two
0,AL,Autauga County,01001,01,001,335,348,348,367,288,304,278,242,282,290,291,294,313,384,394,559,769,697,0500000US01001,"Autauga County, AL"
1,AL,Baldwin County,01003,01,003,1986,2101,1975,1721,1505,1548,1553,1594,1532,1575,1735,1858,2114,2104,2317,2697,3392,3676,0500000US01003,"Baldwin County, AL"
2,AL,Barbour County,01005,01,005,138,179,156,139,133,107,101,120,121,120,129,122,114,113,135,230,339,253,0500000US01005,"Barbour County, AL"
3,AL,Bibb County,01007,01,007,118,145,117,98,98,106,72,95,83,101,78,86,79,105,86,165,179,154,0500000US01007,"Bibb County, AL"
4,AL,Blount County,01009,01,009,315,323,297,306,247,247,263,228,225,228,218,254,281,299,312,344,464,414,0500000US01009,"Blount County, AL"


In [17]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('BFS_CountyAnnual_2005to2022', conn, if_exists = 'replace', index = False)

3156